In [ ]:
import csv

filename = "assignment_3_dataset.csv"

with open(filename, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    print("Headers:", header)
    
    # Print first 5 rows
    for i, row in enumerate(reader):
        print(row)
        if i == 4:  # Stop after 5 rows
            break

In [13]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('assignment_3_dataset.csv')

# Take first 10 rows (you can change this number)
summary_df = df.head(10)

# Plot table
fig, ax = plt.subplots(figsize=(12, 4))  # adjust size as needed
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc='center')

plt.show()


ModuleNotFoundError: No module named 'pandas'

In [ ]:
import sys
print(sys.executable)
